In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
from scipy import ndimage as ndi
import skimage
from skimage import io, filters, morphology
import pandas as pd
import glob
import random
import seaborn as sns
from scipy import stats
from scipy.signal import chirp, find_peaks, peak_widths
from pathlib import Path
import scipy.stats
import os
from scipy import ndimage
import math
from operator import itemgetter
from matplotlib import pyplot
#from read_roi import read_roi_zip
from scipy.optimize import curve_fit
from scipy import signal
from scipy.interpolate import interp1d, splev, CubicSpline, PPoly
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D


In [2]:
#define functions

In [3]:
def display(im3d, cmap="gray", step=1, chosen = None):
    if chosen is None: 
        chosen = []
    stack_size = im3d.shape[0]
    cols = 5
    rows = math.ceil((stack_size/cols))
    
    h , axes = plt.subplots(nrows=rows, ncols=cols, figsize=(3*cols, 3*rows))
    #print(h)
    #p1 = get(h,'Position')
    #annotation('rectangle',p1,'FaceAlpha',.2,'FaceColor','red','EdgeColor','red') 
    vmin = im3d.min()
    vmax = im3d.max()
    enum = 0
    for ax, image in zip(axes.flatten(), im3d[::step]):
        ax.imshow(image, cmap=cmap, vmin=vmin, vmax=vmax)
        if enum in chosen:
            for axis in ['top','bottom','left','right']:
                #ax.spines[axis].set_linewidth(0.5)
                ax.spines[axis].set_color("red")
                ax.spines[axis].set_linewidth(5)
        ax.set_xticks([])
        ax.set_yticks([])
        enum += 1
    #plt.savefig('foo.png')
    

In [4]:
def display_slice(im, cmap="gray", step=1, chosen = None, savefig = False):
    h , axes = plt.subplots(1,1)
    vmin = im.min()
    vmax = im.max()
    enum = 0
    axes.imshow(im, cmap=cmap, vmin=vmin, vmax=vmax)

In [5]:
#functions to get the peak indexes for a particular channel

#get indexes of peaks for edu and hoechst channels in an array, the midpoint is the sum of the peak indexes divided by 2. Can be then converted into a distance.
def get_index(a, b):
    return (a+b)/2

#get the index of a single channel
def get_index_single_axis(a):
    return (a)

#get indexes of peaks for split smc4 or scc1 peaks for subsequent alignment the midpoint is the sum of the peak indexes divided by 2. Can be then converted into a distance.

def get_index_centre_split_axes(a,b):
    return (a+b)/2

In [6]:
#change condition for wapl sor dep2, had to analyse the profiles in batches because the data requirements were too high

In [7]:
wapl_sor_dep_g2 = pd.read_csv("/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/sep_around_axes_Wapl + Sororin depleted G2.csv")

In [8]:
wapl_sor_dep_g2["condition"] = "Wapl + Sororin depleted G2"
wapl_sor_dep_g2.to_csv("/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/sep_around_axes_Wapl + Sororin depleted G2.csv")

In [9]:
csvpath = "/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/"
os.chdir(csvpath)

csv_files = glob.glob(csvpath + "*.csv")

### Fetch all files in path


In [10]:
csv_files


['/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/sep_around_axes_Wapl depleted G2.csv',
 '/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/sep_around_axes_WT late prophase.csv',
 '/groups/gerlich/experiments/Experiments_005500/005556/final_paper_plot_profiles/sep_around_axes_Wapl + Sororin depleted G2.csv']

In [11]:
#create empty list
all_df = []
#read in files
for file in csv_files:
    df = pd.read_csv(file, header=0)
    #append to list
    all_df.append(df)

#concat dfs
frame = pd.concat(all_df, axis=0, ignore_index=True)

#assign variables
difference_edu_scc1_peak = frame["difference_edu_scc1_peak"]

# if difference_edu_scc1_peak negative, displaced outwards relative to the closest SMC peak. if difference positive, displaced inwards relative to the closest SMC peak. This means at the moment that all Wapl values are negative (all displaced outwards because of how I drew the lines) >>> want to create a new column 
# and invert the sign for all values in the column for the final figure

# so that positive values become outwardly displaced, negative values show inward displacement


final_output = frame[["name", "condition", "sister_separation", "difference_edu_scc1_peak"]]

In [14]:
final_output.to_csv(csvpath + "check.csv")

In [19]:
#create empty list
all_df = []
#read in files
for file in csv_files:
    df = pd.read_csv(file, header=0)
    #append to list
    all_df.append(df)

#concat dfs
frame = pd.concat(all_df, axis=0, ignore_index=True)

#assign variables
difference_edu_scc1_peak = frame["difference_edu_scc1_peak"]

# if difference_edu_scc1_peak negative, displaced outwards relative to the closest SMC peak. if difference positive, displaced inwards relative to the closest SMC peak. This means at the moment that all Wapl values are negative (all displaced outwards because of how I drew the lines) >>> want to create a new column 
# and invert the sign for all values in the column for the final figure

# so that positive values become outwardly displaced, negative values show inward displacement

displacement_outward_edu_smc = []
for i in difference_edu_scc1_peak:
    i = -i
    displacement_outward_edu_smc.append(i)
frame["displacement_outward_edu_smc"] = displacement_outward_edu_smc
final_output = frame[["name", "condition", "sister_separation", "displacement_outward_edu_smc"]]

In [20]:
#final_output

In [26]:
results_dir = "/groups/gerlich/experiments/Experiments_005500/005556/filtered_dataframes_for_plotting/"

In [27]:
df_late_prophase

,name,condition,sister_separation,displacement_outward_edu_smc
110,Scc1_EdU_Hoechst_220405_wt_prophase_rep1_hemi_...,WT late prophase,0.554646,-0.116768
111,Scc1_EdU_Hoechst_220405_wt_prophase_rep1_hemi_...,WT late prophase,0.370303,-0.028485
112,Scc1_EdU_Hoechst_220405_wt_prophase_rep1_hemi_...,WT late prophase,0.615273,-0.017091
113,Scc1_EdU_Hoechst_220405_wt_prophase_rep1_hemi_...,WT late prophase,0.474707,0.015313
114,Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_propha...,WT late prophase,0.364606,-0.113939
...,...,...,...,...
262,Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_propha...,WT late prophase,0.587626,-0.117525
263,Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_propha...,WT late prophase,0.661141,0.068394
264,Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_propha...,WT late prophase,0.564000,-0.068364
265,Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_propha...,WT late prophase,0.611000,0.018515


In [37]:
df_late_prophase = final_output.loc[final_output['condition'] == "WT late prophase"]
df_late_prophase.to_csv(results_dir + "late_prophase_output.csv", index=False)

df_wapl = final_output.loc[final_output['condition'] == "Wapl depleted G2"]
df_wapl.to_csv(results_dir + "wapl_dep_output.csv", index=False)

df_wapl_sor = final_output.loc[final_output['condition'] == "Wapl + Sororin depleted G2"]
df_wapl_sor.to_csv(results_dir + "wapl_sor_dep_output.csv", index=False)


In [42]:
sorted_prophase = df_late_prophase.groupby('name')['sister_separation'].mean()

In [43]:
sorted_prophase

name
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #1.tif_registered.tif    0.640960
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #2.tif_registered.tif    0.571111
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #4.tif_registered.tif    0.600402
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #5.tif_registered.tif    0.593697
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #7.tif_registered.tif    0.445993
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep1_prophase_smc4_axes_60min_stlc_hemi_zoom5-01-46.czi #8.tif_registered.tif    0.604044
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep2_prophase_axes_60min_stlc_hemi_zoom5-01-60.czi #1.tif_registered.tif         0.540841
Scc1_EdU_Hoechst_220207_5389_WT_c1_rep2_prophase_axes_60min_stlc_hemi_zoom5-01-60.czi #3.tif_registered.tif         0.58

In [45]:
count_row = sorted_prophase.shape[0]
count_row

31

In [46]:
sorted_wapl= df_wapl.groupby('name')['sister_separation'].mean()

In [47]:
count_row = sorted_wapl.shape[0]
count_row

18

In [48]:
sorted_wapl

name
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep1_wapl_dep_on_hemi_g2_zoom4_8-03-51.tif_registered.tif               0.398808
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep1_wapl_dep_on_hemi_g2_zoom4_8-06-40.czi #1.tif_registered.tif        0.369734
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep1_wapl_dep_on_hemi_g2_zoom4_8-06-40.czi #2.tif_registered.tif        0.363573
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep1_wapl_dep_on_hemi_g2_zoom4_8-07-41-43.tif_registered.tif            0.404726
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep1_wapl_dep_on_hemi_g2_zoom4_8-09-45.tif_registered.tif               0.368419
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_dep_on_hemi_g2_zoom4_8-03-52.tif_registered.tif               0.398107
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_dep_on_hemi_g2_zoom4_8-09-55.tif_registered.tif               0.415985
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_dep_on_hemi_g2_zoom4_8-13-58.czi #1.tif_registered.tif        0.439983
Scc1_EdU_Hoechst_220211_5389_2096_c2_rep1_wapl_dep_

In [52]:
sorted_wapl_sor= df_wapl_sor.groupby('name')['sister_separation'].mean()

In [53]:
count_row = sorted_wapl_sor.shape[0]
count_row

18

In [54]:
sorted_wapl_sor

name
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_sor_dep_on_hemi_g2_zoom4_8-03-37.czi #2.tif_registered.tif        0.731798
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_sor_dep_on_hemi_g2_zoom4_8-03-37.czi #5.tif_registered.tif        0.614880
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_sor_dep_on_hemi_g2_zoom4_8-03-37.czi #6.tif_registered.tif        0.693044
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_sor_dep_on_hemi_g2_zoom4_8-03-37.czi #7.tif_registered.tif        0.636834
Scc1_EdU_Hoechst_220211_5389_2096_c1_rep2_wapl_sor_dep_on_hemi_g2_zoom4_8-03-37.czi #8.tif_registered.tif        0.639887
Scc1_EdU_Hoechst_220211_5389_2096_c2_rep1_wapl_sor_dep_on_hemi_g2_zoom4_8-02-38.czi #1.tif_registered.tif        0.611318
Scc1_EdU_Hoechst_220211_5389_2096_c2_rep1_wapl_sor_dep_on_hemi_g2_zoom4_8-02-38.czi #2.tif_registered.tif        0.600112
Scc1_EdU_Hoechst_220211_5389_2096_c2_rep1_wapl_sor_dep_on_hemi_g2_zoom4_8-02-38.czi #3.tif_registered.tif        0.739205
Scc1_EdU_Hoechst_22